In [1]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool
from pydantic import BaseModel

In [2]:
# Define the LLM
llm = ChatOpenAI(model='gpt-4o', temperature=0.5)

In [3]:
brand_info = {
    "name": "Zero to Mastery",
    "description": "An online tech academy that offers comprehensive courses and a community for learners seeking in-demand software and tech skills.",
    "brand_values": "Quality education, community-driven growth, career-focused learning, and up-to-date tech stacks.",
    "ideal_partner": "A YouTube channel focused on programming, software engineering, or tech career advice with an audience of motivated learners."
}

In [4]:
# Define a single YouTube lead for demonstration
youtube_lead = {
    "name": "Theo - t3․gg",
    "channel_url": "https://www.youtube.com/@t3dotgg"
}

In [5]:
research_agent = Agent(
    role="YouTube Research Specialist",
    goal=f"""
        Gather relevant information about {youtube_lead['name']}’s channel {youtube_lead['channel_url']} to assess suitability for {brand_info['name']}.
        """,
    backstory=f"""
        You are responsible for checking if the channel aligns with {brand_info['name']}'s core values: {brand_info['brand_values']}.
        """,
    llm=llm,
    max_iter=3,
    tools=[YoutubeChannelSearchTool(), ScrapeWebsiteTool(), SerperDevTool(), YoutubeVideoSearchTool()]
)

In [6]:
research_task = Task(
    description=f"""
        Research the channel "{youtube_lead['name']}" at {youtube_lead['channel_url']}:
        1. Identify channel focus and content type
        2. Determine approximate subscriber count and engagement
        3. Check for audience alignment with {brand_info['name']}
    """,
    expected_output="A concise research report on the channel’s key attributes and audience.",
    agent=research_agent
)

In [7]:
video_research_task = Task(
    description=f"""
        Pick 2 recent or popular videos from the channel {youtube_lead['channel_url']}, summarize their topic, and analyze audience response (views, comments, sentiment).
    """,
    expected_output="Short summaries of the videos’ topics and engagement metrics (views, comments, sentiment).",
    agent=research_agent
)

In [8]:
sentiment_task = Task(
    description=f"""
        Perform a quick sentiment analysis by searching Google for external mentions of {youtube_lead['name']} or their channel {youtube_lead['channel_url']}.
        Look for user feedback, reviews, or discussions that indicate general sentiment about the channel.
    """,
    expected_output="A summary of external sentiment around the channel, if any.",
    agent=research_agent
)

In [9]:
class Score(BaseModel):
    score: int

In [10]:
scoring_agent = Agent(
    role="Partnership Scoring Agent",
    goal=f"""
        Evaluate how well {youtube_lead['name']} aligns with {brand_info['name']}, focusing on brand values:
        {brand_info['brand_values']}.
    """,
    backstory = f"You are responsible for checking if the channel aligns with {brand_info['name']}'s core values: {brand_info['brand_values']}",
    max_iter=3,
    llm=llm
)

In [11]:
scoring_task = Task(
    description="Review the research findings and assign a score from 0 to 10 for potential partnership suitability.",
    expected_output="""
    A numeric score and brief justification for the rating.
    An explanation of the score indicating what fits the brand and what does not
    """,
    agent=scoring_agent,
    context = [research_task, video_research_task, sentiment_task],
    output_pydantic=Score
)

In [12]:
crew = Crew(
    agents=[research_agent, scoring_agent],
    tasks=[research_task, video_research_task, sentiment_task, scoring_task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d35d044c-d59a-42e3-8b49-da2cb1bea764                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: YouTube Research Specialist                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Research the channel "Theo - t3․gg" at https://www.youtube.com/@t3dotgg:                               │
│          1. Identify channel focus and content type                                                             │
│          2. Determine approximate subscriber count and engagement                                               │
│          3. Check for audience alignment with Zero to Mastery                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/dylanthunn/.local/share/virtualenvs/agents-0RAN7L6M/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


/Users/dylanthunn/.local/share/virtualenvs/agents-0RAN7L6M/lib/python3.12/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Search a Youtube Channels content                                                                        │
│  Error: Invalid YouTube channel URL: @t3dotgg                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

/Users/dylanthunn/.local/share/virtualenvs/agents-0RAN7L6M/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Partnership Scoring Agent                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "score": 9                                                                                                   │
│  }                                                                                                              │
│                                                                                                                 │
│  The channel "Theo - t3․gg" aligns well with Zero to Master's core values, warranting a high partnership        │
│  suitability score. It excels in providing quality education through in-depth discussions and tutorials on      │
│  software development and AI, which is crucial for learners. The channel fosters community-driven growth by     │
│  engaging with the tech community and encouraging discussions, aligning with Zero to Master's emphasis on       │
│  community. Its career-focused content, offering industry insights and career advice, supports viewers in       │
│  advancing their careers, which aligns with Zero to Master's learning goals. Additionally, the channel's focus  │
│  on up-to-date tech stacks, particularly in AI and TypeScript, ensures that it covers current industry          │
│  standards, which is critical for keeping learners informed about technological advancements. Despite some      │
│  mixed sentiment regarding specific content and presentation, the channel's overall positive impact and strong  │
│  following within the tech community make it a suitable partner for Zero to Mastery.                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ae82072b-53a1-44c5-b43c-afcf84eb65bc                                                                     │
│  Agent: Partnership Scoring Agent                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d35d044c-d59a-42e3-8b49-da2cb1bea764                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "score": 9                                                                                                   │
│  }                                                                                                              │
│                                                                                                                 │
│  The channel "Theo - t3․gg" aligns well with Zero to Master's core values, warranting a high partnership        │
│  suitability score. It excels in providing quality education through in-depth discussions and tutorials on      │
│  software development and AI, which is crucial for learners. The channel fosters community-driven growth by     │
│  engaging with the tech community and encouraging discussions, aligning with Zero to Master's emphasis on       │
│  community. Its career-focused content, offering industry insights and career advice, supports viewers in       │
│  advancing their careers, which aligns with Zero to Master's learning goals. Additionally, the channel's focus  │
│  on up-to-date tech stacks, particularly in AI and TypeScript, ensures that it covers current industry          │
│  standards, which is critical for keeping learners informed about technological advancements. Despite some      │
│  mixed sentiment regarding specific content and presentation, the channel's overall positive impact and strong  │
│  following within the tech community make it a suitable partner for Zero to Mastery.                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
# Access the final outputs
channel_research_output = result.tasks_output[0].raw
video_research_output = result.tasks_output[1].raw
sentiment_output = result.tasks_output[2].raw
scoring_output = result.tasks_output[3].raw

print("=== YouTube Channel Research ===")
print(channel_research_output)

print("\n=== Video Research ===")
print(video_research_output)

print("\n=== Sentiment Analysis ===")
print(sentiment_output)

print("\n=== Partnership Score ===")
print(scoring_output)

=== YouTube Channel Research ===
**Theo - t3․gg YouTube Channel Research Report**

1. **Channel Focus and Content Type:**
   - Theo - t3․gg is a YouTube channel focused on software development, AI, and TypeScript. The channel is run by Theo Browne, who is known for creating T3 Chat and the T3 Stack.
   - Content includes discussions on the latest in AI, software development best practices, and insights into the tech industry. Theo also shares personal experiences and advice related to tech and startups.

2. **Subscriber Count and Engagement:**
   - The channel has approximately 492K subscribers, indicating a strong following within the tech community.
   - The channel has amassed over 87 million total views, demonstrating significant engagement and reach.
   - Theo - t3․gg frequently uploads content, with a current total of around 930 videos.

3. **Audience Alignment with Zero to Mastery:**
   - The channel aligns well with Zero to Mastery's core values:
     - **Quality Education:** T